In [1]:
!pip install opencv-python

In [95]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

Project_Path = "C:\\Users\\Othmane Tazi\\Desktop\\PgProject"
crop_data_path = os.path.join(Project_Path, 'crop_coordinates.csv')
input_images_path = os.path.join(Project_Path, 'KinectOutput/Images')
output_images_path = os.path.join(Project_Path, 'FormattedImages')

In [108]:
# function to apply the edge highlighting technique
def format_hand_image(hand_, x_max, x_min, y_max, y_min):
    # calculate crop
    x_dist = x_max - x_min
    y_dist = y_max - y_min
    
    dist = max(x_dist, y_dist)
    
    x_min = max(x_min - 20, 0)
    y_min = max(y_min - 20, 0)
    
    x_max = min(x_min + dist + 40, 1920)
    y_max = min(y_min + dist + 40, 1080)
    
    # apply crop
    hand_ =  hand_[y_min:y_max, x_min:x_max]
    
    #apply gaussian blurr
    kernel_size2 = (5, 5)
    hand_blurr = cv2.GaussianBlur(hand_, kernel_size2, 0)

    # turn to grayscale
    hand_gray = cv2.cvtColor(hand_blurr, cv2.COLOR_RGB2GRAY)

    # Apply Sobel edge detection in horizontal direction
    sobel_x = cv2.Sobel(hand_gray, cv2.CV_64F, 1, 0, ksize=3)

    # Apply Sobel edge detection in vertical direction
    sobel_y = cv2.Sobel(hand_gray, cv2.CV_64F, 0, 1, ksize=3)

    # Combine the horizontal and vertical edge images
    hand_edges = cv2.addWeighted(np.absolute(sobel_x), 0.5, np.absolute(sobel_y), 0.5, 0)

    #apply thresholding
    _, hand_thrshld = cv2.threshold(hand_edges, 31,255,cv2.THRESH_BINARY)
    
    resized_image = cv2.resize(hand_thrshld, (256, 256))
    
    return resized_image

In [112]:
# get croping data
crop_coord = np.genfromtxt(crop_data_path, delimiter=',', dtype=int)
indecies = crop_coord[:, 0].astype(int)

crop_coord.shape

(1475, 5)

In [115]:
# apply the Edge highliting technique on the images and save them
for info in crop_coord:
    color_img = cv2.imread(os.path.join(input_images_path, str(info[0])+'.png'))
    formated_image = format_hand_image(color_img, info[1], info[2], info[3], info[4])
    img_path =  os.path.join(output_images_path, str(info[0])+'.png')
    cv2.imwrite(img_path, formated_image)
    
    